# Camera Calibration

- Unknown : Intrinsic + extrinsic parameters (5* + 6 DoF)
    - The number of intrinsic parameters* can be varied w.r.t. user preference.
- Given : 3D Points $X_1, X_2, \dots, X_n$ and their projected points $x_1, x_2, \dots,x_n$
- Constraints : n x projection $x_i = K[R|t]X_i$
- Solution
    - OpenCV `cv::calibrateCamera()` and `cv::initCameraMatrix2D()`
    - Camera Calibration Toolbox for MATLAB, http://www.vision.caltech.edu/bouguetj/calib_doc/
    - GML C++ Camera Calibration Toolbox, http://graphics.cs.msu.ru/en/node/999
    - DrakCamCalibrator, http://darkpgmr.tistory.com/139


In [1]:
import math
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline

### Why is the checkerboard pattern so widely used in calibration?
Checkerboard patterns are distinct and easy to detect in an image. Not only that, the corners of squares on the checkerboard are ideal for localizing them because they have sharp gradients in two directions. In addition, these corners are also related by the fact that they are at the intersection of checkerboard lines. All these facts are used to robustly locate the corners of the squares in a checkerboard pattern. [learnopencv](https://learnopencv.com/camera-calibration-using-opencv/)


In [2]:
# Load and display a .mp4 video.
from IPython.display import HTML
from base64 import b64encode
input_name = "chessboard.mp4"
input = open(input_name,'rb').read()
input_url = "data:video/mp4;base64," + b64encode(input).decode()

In [3]:
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % input_url)

Output hidden; open in https://colab.research.google.com to view.

In [61]:
# ChessBoard Setting
board_pattern = (10, 7)

# Open a Video
cap = cv2.VideoCapture(input_name)
if cap.isOpened() == False: raise Exception("No video")

# Get some Chessboard images
images = []
displays = []
count = 0
while True:
    ret, image = cap.read()
    if not ret: break

    ret, pts = cv2.findChessboardCorners(image, board_pattern, None) # No flags
    display = copy.deepcopy(image)
    display = cv2.drawChessboardCorners(display, board_pattern, pts, ret)
    if count % 24 == 0: # 1 second of video roughly has 24 images.
      images.append(image)
      displays.append(display)
    count += 1
cap.release()

if not len(images): raise Exception("No images")
ncols = 3
nrows = math.ceil(len(images) / ncols)
f, ax = plt.subplots(nrows, figsize=[200, 60])
for i in range(nrows):
  vis_image = np.hstack((displays[i*ncols:(i+1)*ncols]))
  ax[i].imshow(vis_image[:, :, (2, 1, 0)])
  ax[i].axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [62]:
# Find 2D corner Points from given images
img_points = []
for image in images:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    ret, corners = cv2.findChessboardCorners(gray, board_pattern) # No flags
    if ret == True:
        img_points.append(corners)

if len(img_points) == 0:
    raise Exception("No 2D corner points")

len(img_points), img_points[0].shape #, img_points[0]

(67, (70, 1, 2))

In [63]:
# Prepare 3D Points of the Chessboard
objp = np.zeros((board_pattern[0]*board_pattern[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:board_pattern[0], 0:board_pattern[1]].T.reshape(-1, 2)
obj_points = [objp for _ in range(len(images))]

len(obj_points), obj_points[0].shape #, obj_points[0]

(67, (70, 3))

In [64]:
# Calibrate Camera
rms, K, dist_coeff, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, (h, w), None, None)

# Report calibration results
print("## Camera Calibration Results")
print(f"* The number of applied images = {w}x{h}")
print(f"* RMS error = {rms}")
print(f"* Camera matrix (K) = \n{K}")
print(f"* Distortion coefficient (k1, k2, p1, p2, k3, ...)\n = {dist_coeff}")

## Camera Calibration Results
* The number of applied images = 960x540
* RMS error = 0.38628125467759156
* Camera matrix (K) = 
[[433.4245397    0.         476.08725257]
 [  0.         432.01861416 289.39480035]
 [  0.           0.           1.        ]]
* Distortion coefficient (k1, k2, p1, p2, k3, ...)
 = [[-2.89565055e-01  1.06364427e-01 -4.08941688e-04  6.73218603e-05
  -1.95185541e-02]]
